# RFM Analysis

This iPython notebook explains how to perform RFM analysis from customer purchase history data. The sample orders file is Sample - Superstore dataset from Tableau Software.

If you have suggestions or improvements please contribute on https://github.com/joaolcorreia/RFM-analysis

In [1]:
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Read the sample orders file, containing all past purchases for all customers.

In [2]:
orders = pd.read_csv('DETALLE_DE_PEDIDOS.csv', sep=';', encoding='latin-1')

In [3]:
orders['order_date'] = pd.to_datetime (orders['FechaPedido']+ orders['Hora Pedido'] , format = '%d-%m-%Y%H:%M:%S')
orders['order_date'] = orders.order_date.dt.strftime('%m/%d/%Y %H:%M:%S')

In [4]:
orders.dtypes

FechaPedido                        object
Hora Pedido                        object
PedidoFechaDia                      int64
PedidoFechaMesNumero                int64
PedidoFechaAño                      int64
Tipo Negocio                       object
Rut                                object
Nombre_Razon_Social                object
Apellido_Nombre_Fantasia           object
Pedido_Id                           int64
Numero_Pedido_Origen               object
Codigo_Autorizacion               float64
Numero_Boleta                       int64
Comuna                             object
SKU                                object
Precio Original Unitario Bruto     object
Precio Final Unitario Bruto        object
Suma de Cantidad SKU                int64
Venta Total Original Bruto         object
Venta Total Final Bruto            object
Nombre Producto                    object
Tipo Documento                     object
Courier                            object
Estado Actual Pedido              

## Create the RFM Table

Since recency is calculated for a point in time and the Tableau Super Store dataset last order date is Dec 31 2014, that is the date we will use to calculate recency.

Set this date to the current day and extract all orders until yesterday.

In [5]:
NOW = datetime.now()

In [6]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

Create the RFM Table

In [7]:
rfmTable = orders.groupby('Rut').agg({'order_date': lambda x: (NOW - x.max()).days, # Recency
                                        'Pedido_Id': lambda x: len(x),      # Frequency
                                        'TotalNeto': lambda x: x.sum()}) # Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'Pedido_Id': 'frequency', 
                         'TotalNeto': 'monetary_value'}, inplace=True)

## Validating the RFM Table

In [8]:
rfmTable.head()

,recency,frequency,monetary_value
Rut,,,
10071410-8,39,2,18.597
10140515-K,39,2,26.660
10143072-3,167,3,33.185
10195590-7,145,8,217.280
10199929-7,60,2,23.679


Customer **Aaron Bergman** has frequency:3, monetary value:$887 and recency:415 days.

In [9]:
Paola = orders[orders['Rut']=='15425478-1']
Paola

,FechaPedido,Hora Pedido,PedidoFechaDia,PedidoFechaMesNumero,PedidoFechaAño,Tipo Negocio,Rut,Nombre_Razon_Social,Apellido_Nombre_Fantasia,Pedido_Id,...,Corrimiento,Email,Telefono,Direccion,Dpto,Suma de MontoOriginalNeto,Suma de DescuentoNeto,TotalNeto,Nombre Completo,order_date
2,16-10-2019,0:45:52,16,10,2019,Venta Web Cotidian,15425478-1,paola,oyarzun,1946,...,NaN,paolaoyarzuna@gmail.com,56978561858,"Los Gladiolos 3477, Puente Alto, CHILE",NaN,118.891,31.664,87.227,paola oyarzun,2019-10-16 00:45:52
3,16-10-2019,0:45:52,16,10,2019,Venta Web Cotidian,15425478-1,paola,oyarzun,1946,...,NaN,paolaoyarzuna@gmail.com,56978561858,"Los Gladiolos 3477, Puente Alto, CHILE",NaN,3.353,3.353,0.000,paola oyarzun,2019-10-16 00:45:52
882,28-08-2019,21:12:02,28,8,2019,Venta Web Cotidian,15425478-1,paola,oyarzun,1226,...,-14.0,paolaoyarzuna@gmail.com,56978561858,"Los Gladiolos 3477, Puente Alto, CHILE",NaN,89.168,29.720,59.448,paola oyarzun,2019-08-28 21:12:02
883,28-08-2019,21:12:02,28,8,2019,Venta Web Cotidian,15425478-1,paola,oyarzun,1226,...,-14.0,paolaoyarzuna@gmail.com,56978561858,"Los Gladiolos 3477, Puente Alto, CHILE",NaN,0.000,0.000,0.000,paola oyarzun,2019-08-28 21:12:02


Inserting the date of Aaron purchase and comparing it to the recency in the rfmTable we verify our RFM table is correct.

## Determining RFM Quartiles

In [10]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [11]:
quantiles

,recency,frequency,monetary_value
0.25,13.0,2.0,27.8885
0.50,39.0,3.0,46.2860
0.75,117.5,4.0,85.0140


Send quantiles to a dictionary, easier to use.

In [12]:
quantiles = quantiles.to_dict()

In [13]:
quantiles

{'recency': {0.25: 13.0, 0.5: 39.0, 0.75: 117.5},
 'frequency': {0.25: 2.0, 0.5: 3.0, 0.75: 4.0},
 'monetary_value': {0.25: 27.8885, 0.5: 46.286, 0.75: 85.014}}

## Creating the RFM segmentation table

In [14]:
rfmSegmentation = rfmTable

In [15]:
rfmSegmentation['monetary_value']= round( rfmSegmentation['monetary_value'],9)

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [16]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [17]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [18]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [19]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
Rut,,,,,,,
10071410-8,39,2,18.597,2,4,4,244
10140515-K,39,2,26.660,2,4,4,244
10143072-3,167,3,33.185,4,3,3,433
10195590-7,145,8,217.280,4,1,1,411
10199929-7,60,2,23.679,3,4,4,344


In [20]:
# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
# rfmSegmentation.to_clipboard()
# rfmSegmentation.to_csv('rfm-table.csv', sep=',')

Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently?

In [21]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
Rut,,,,,,,
7087506-3,13,6,530.740,1,1,1,111
4546453-9,12,7,489.764,1,1,1,111
6374703-3,13,18,414.630,1,1,1,111
10492007-1,10,8,349.427,1,1,1,111
7984447-0,12,12,334.997,1,1,1,111


In [22]:
import sqlite3
con = sqlite3.connect(':memory:')
rfmSegmentation.to_sql('Segmentation', con)

In [23]:
resumen  = pd.read_sql_query('SELECT RFMClass, COUNT(Rut) AS Clientes FROM Segmentation GROUP BY RFMClass',con)
resumen

,RFMClass,Clientes
0,111,33
1,112,11
2,113,5
3,121,13
4,122,7
5,123,5
6,124,4
7,131,5
8,132,11
9,133,10


In [24]:
def Segment(x):
    if x <= 222:
        return 1
    elif x <= 344:
        return 2
    else:
        return 3

In [25]:
rfmSegmentation['RFMClass2'] = rfmSegmentation['RFMClass'].astype(int)
#rfmSegmentation['Segmento'] = rfmSegmentation['RFMClass2'].apply(Segment, args=('RFMClass2',quantiles,))

In [26]:
rfmSegmentation['Segmento'] = rfmSegmentation['RFMClass2'].apply(Segment)

In [27]:
import sqlite3
con = sqlite3.connect(':memory:')
rfmSegmentation.to_sql('Segmentation2', con)

resumen  = pd.read_sql_query('SELECT Segmento, COUNT(Rut) AS Clientes FROM Segmentation2 GROUP BY Segmento',con)
resumen

,Segmento,Clientes
0,1,247
1,2,175
2,3,141


In [28]:
rfmSegmentation.to_csv('Clientes_Segmentados.csv')